### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import urllib.parse
import re

In [ ]:
# Set environment variables from the .env in the local environment

dotenv_path = 'API_Key.env'
load_dotenv(dotenv_path)

In [ ]:

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(nyt_api_key)
print(tmdb_api_key)

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Initial simpler query to test if we get any results
#filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# Use simpler queries to debug
filter_queries = [
    'section_name:"Movies"',
    'type_of_material:"Review"',
    'headline:"love"'
]

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Test each filter query individually
for filter_query in filter_queries:
    encoded_filter_query = urllib.parse.quote(filter_query)
    # Build URL
    query_url = (
        f"{url}"
        f"api-key={nyt_api_key}&"
        f"fq={encoded_filter_query}&"
        f"sort={sort}&"
        f"fl={field_list}&"
        f"begin_date={begin_date}&"
        f"end_date={end_date}&"
        f"page=0"
    )
    # Make a request to test the URL
    response = requests.get(query_url)
    print(f"Testing filter query: {filter_query}")
    print(response.url)
    print(f"Page 0 response status code: {response.status_code}")
    print(response.json())
    print("\n")


In [ ]:
    # Make a request to test the URL
    response = requests.get(query_url)
    print(f"Testing filter query: {filter_query}")
    print(response.url)
    print(f"Page 0 response status code: {response.status_code}")
    print(response.json())
    print("\n")

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

In [ ]:

# loop through pages 0-19
for page in range(0, 20):
    query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&page={page}"
   
    # create query with a page number
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()  
# Print the response status code and a part of the response content for debugging
    print(f"Page {page} response status code: {response.status_code}")
    if response.status_code != 200:
        print(f"Error: {response.json()}")
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    # loop through the reviews["response"]["docs"] and append each review to the list
    # Print the page that was just retrieved
    # Print the page number that had no results then break from the loop  
    try:
        docs = reviews.get("response", {}).get("docs", [])
        if not docs:
            print(f"No results found on page {page}")
            break
        for doc in docs:
            reviews_list.append(doc)
        print(f"Page {page} processed")
    except Exception as e:
        print(f"An error occurred on page {page}: {e}")
        break

    



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = pd.json_normalize(reviews_list)
df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
def extract_title(headline):
    match = re.search(r'\u2018(.*?)\u2019', headline)
    if match:
        title = match.group(1) + " Review"
        return title
    else:
        return None
    
df['title'] = df['headline.main'].apply(extract_title)


df.head()  

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
# def extract_keywords(keyword_list):
#     extracted_keywords = ""
#     for item in keyword_list:
#         # Extract 'name' and 'value'
#         keyword = f"{item['name']}: {item['value']};" 
#         # Append the keyword item to the extracted_keywords list
#         extracted_keywords += keyword
#     return extracted_keywords

# # Fix the "keywords" column by converting cells from a list to a string
# df['keywords'] = df['keywords'].apply(extract_keywords)
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        if isinstance(item, dict):
            keyword = f"{item.get('name', '')}: {item.get('value', '')}; "
            extracted_keywords += keyword
    return extracted_keywords.strip()  # Strip trailing space

# Fix the "keywords" column
df['keywords'] = df['keywords'].apply(extract_keywords)

In [ ]:
df.head()

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
nyt_titles = df['title'].tolist()
print(nyt_titles)

In [ ]:
# # Filter out None values and remove " Review" suffix
cleaned_titles = [title.replace(" Review", "") for title in nyt_titles if title is not None]

# # Print the cleaned titles
print(cleaned_titles)

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
tmdb_url

In [ ]:
# Create an empty list to store the results
tmdb_list = []

In [ ]:
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


In [ ]:
# Loop through the titles
for title in cleaned_titles:
    try:
        tmdb_query_url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
     # Perform a "GET" request for The Movie Database
        tmdb_response = requests.get(tmdb_query_url)
            # Add 1 to the request counter
    # Make the request to TMDB
   
        request_counter += 1
# Check if we need to sleep before making a request
        if request_counter % 50 == 0:
            print(f"API rate limit - (Request {request_counter})")
            time.sleep(10)
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        if tmdb_response.status_code == 200:
            tmdb_list.append(tmdb_response.json())
        else: 
            print(f"No movie found for title {title}")
    except Exception as e:
        print(f"An error occurred searching for: {title}")
        print(f"Error details: {str(e)}")

In [ ]:
tmdb_list

In [ ]:

        # Get movie id
        if tmdb_response.status_code == 200:
            tmdb_data = tmdb_response.json()

            # Check if the movie was found
            if tmdb_data['results']:
                movie_id = tmdb_data['results'][0]['id']
                # Make a request for the full movie details
                tmdb_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
                tmdb_movie_response = requests.get(tmdb_movie_url)

                if tmdb_movie_response.status_code == 200:
                    movie_details = tmdb_movie_response.json()

                    # Extract the genre names into a list
                    genre_names = [genre['name'] for genre in movie_details.get('genres', [])]

                    # Extract the spoken_languages' English name into a list
                    language_names = [language['english_name'] for language in movie_details.get('spoken_languages', [])]

                    # Extract the production_countries' name into a list
                    country_names = [country['name'] for country in movie_details.get('production_countries', [])]

                    # Add the relevant data to a dictionary and append it to the tmdb_list list
                    movie_dict = {
                        'title': movie_details.get('title'),
                        'genres': genre_names,
                        'spoken_languages': language_names,
                        'production_countries': country_names,
                        'release_date': movie_details.get('release_date'),
                        'runtime': movie_details.get('runtime'),
                        'vote_average': movie_details.get('vote_average'),
                        'overview': movie_details.get('overview')
                    }

                    # Append the movie details to the list
                    tmdb_list.append(movie_dict)

                    # Print out the title that was found
                    print(f"Found details for movie: {title}")
                else:
                    print(f"Failed to get details for movie ID {movie_id}")
            else:
                print(f"No movie found for title {title}")
        else:
            print(f"Failed to search for movie: {title} - Status code: {tmdb_response.status_code}")


In [ ]:
movie_id

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_list[:5], indent=4))


In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.json_normalize(tmdb_list)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df, tmdb_df, on='title', how='inner')


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame
print(merged_df.head())

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=['byline.person'])

In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)